In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Layer
import warnings
warnings.filterwarnings('ignore')


class CustomAttention(Layer):
    def __init__(self, attention_dim=64, **kwargs):
        super(CustomAttention, self).__init__(**kwargs)
        self.attention_dim = attention_dim

    def build(self, input_shape):
        self.input_dim = input_shape[-1]
        self.query_dense = tf.keras.layers.Dense(self.attention_dim, activation='tanh')
        self.key_dense = tf.keras.layers.Dense(self.attention_dim, activation='tanh')
        self.value_dense = tf.keras.layers.Dense(self.input_dim, activation='tanh')
        super(CustomAttention, self).build(input_shape)

    def call(self, inputs):
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)
        attention_scores = tf.keras.backend.batch_dot(query, key, axes=[2, 2])
        attention_scores = tf.nn.softmax(attention_scores, axis=-1)
        attended = tf.keras.backend.batch_dot(attention_scores, value, axes=[2, 1])
        attended = attended + inputs

        return attended

    def get_config(self):
        config = super(CustomAttention, self).get_config()
        config.update({"attention_dim": self.attention_dim})
        return config


class WeatherPredictor:
    def __init__(self, model_path='best_model.h5', sequence_length=20):
        self.sequence_length = sequence_length
        self.model = None
        self.weather_labels = {0: 'Sunny', 1: 'Rainy', 2: 'Stormy', 3: 'Windy'}
        self.load_model(model_path)
        self.feature_means = np.array([0.5, 0.5, 0.7, 0.3, 0.5, 0.47, 0.48, 0.4, 0.1])
        self.feature_stds = np.array([0.2, 0.3, 0.15, 0.2, 0.15, 0.02, 0.02, 0.02, 0.1])

    def load_model(self, model_path):
        try:
            self.model = load_model(model_path, custom_objects={'CustomAttention': CustomAttention})
            print(f"Model loaded successfully from {model_path}")
        except Exception as e:
            print(f"Error loading model: {e}")
            print("Make sure the model file exists and the path is correct")

    def parse_input(self, input_string):
        parts = input_string.strip().split(',')
        features = []
        for i in range(1, len(parts)):
            features.append(float(parts[i]))

        return np.array(features)

    def create_sequence(self, single_input):
        sequence = np.tile(single_input, (self.sequence_length, 1))
        return sequence

    def normalize_input(self, features):
        normalized = (features - self.feature_means) / \
            (self.feature_stds + 1e-8)
        return normalized

    def predict_weather(self, input_string):
        if self.model is None:
            print("Model not loaded. Please check the model path.")
            return None

        try:
            features = self.parse_input(input_string)
            print(f"Parsed features: {features}")
            normalized_features = self.normalize_input(features)
            print(f"Normalized features: {normalized_features}")
            sequence = self.create_sequence(normalized_features)
            sequence = sequence.reshape(1, self.sequence_length, -1)
            print(f"Input sequence shape: {sequence.shape}")
            prediction_proba = self.model.predict(sequence, verbose=0)
            predicted_class = np.argmax(prediction_proba, axis=1)[0]
            confidence = np.max(prediction_proba) * 100
            predicted_weather = self.weather_labels[predicted_class]
            print(f"\nPrediction Results:")
            print(f"Predicted Weather: {predicted_weather} (Class {predicted_class})")
            print(f"Confidence: {confidence:.2f}%")
            print(f"All probabilities: {prediction_proba[0]}")

            return predicted_class, predicted_weather, confidence

        except Exception as e:
            print(f"Error during prediction: {e}")
            return None


def main():
    predictor = WeatherPredictor()
    input_data = "16:10:50,0.5238970588235294,0.5066666666666667,0.8872549019607843,0.5685279187817258,0.497716894977169,0.4643410499453153,0.4793204697986576,0.4143035099289221,0.0"
    print("Weather Prediction Script")
    print("=" * 50)
    print(f"Input: {input_data}")
    print("=" * 50)
    result = predictor.predict_weather(input_data)
    if result:
        predicted_class, predicted_weather, confidence = result
        print(f"Final Prediction: {predicted_weather}")
    else:
        print("\nPrediction failed")

if __name__ == "__main__":
    main()

Model loaded successfully from best_model.h5
Weather Prediction Script
Input: 16:10:50,0.5238970588235294,0.5066666666666667,0.8872549019607843,0.5685279187817258,0.497716894977169,0.4643410499453153,0.4793204697986576,0.4143035099289221,0.0
Parsed features: [0.52389706 0.50666667 0.8872549  0.56852792 0.49771689 0.46434105
 0.47932047 0.41430351 0.        ]
Normalized features: [ 0.11948529  0.02222222  1.24836593  1.34263953 -0.0152207  -0.28294736
 -0.03397649  0.71517514 -0.9999999 ]
Input sequence shape: (1, 20, 9)

Prediction Results:
Predicted Weather: Windy (Class 3)
Confidence: 99.21%
All probabilities: [0.00311992 0.00107067 0.00368056 0.99212885]
Final Prediction: Windy


In [1]:
import pandas as pd

df = pd.read_csv("midiMusic.csv", header=None)
df_cleaned = df.iloc[:, :-1]
df_cleaned.to_csv("midiMusic_cleaned.csv", index=False, header=False)